# Utils

> Utils function used in the environment.

In [ ]:
#| default_exp utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import toml

In [ ]:
#| export
def read_toml(  config_file= "Config.toml" # Path to the config file
):
    "Read setting file. The File containes the relative path to source and target."
    config= toml.load(config_file)
    return config
    

In [ ]:
# Here an example on how to structure a config.toml file.
config  = read_toml()
config

{'source': {'path': 'data/source',
  'filename_geometry': 'bone.key',
  'filename_landmarks': 'landmarks_source.csv'},
 'target': {'path': 'data/target',
  'filename_landmarks': 'landmarks_target.key'}}

In [ ]:
#| export
def _merge_path(path1:str , path2: str)-> str:
    "Join one or more path components. Return the join path in str type."
    return os.path.join(path1,path2)

In [ ]:
_merge_path(config['source']['path'],config['source']['filename_geometry'])

'data/source/bone.key'

In [ ]:
#| export
def multiple_targets():
    "This function checks if the there are multiple target subject."
    config = read_toml()
    path= config['target']['path']
    filename = config['target']['filename_landmarks']
    list_target= [name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))]
    if not list_target:
        assert os.path.isfile(_merge_path(path, filename)), "No file with Landmarks"
        return False
    else:
        return list_target


In [ ]:
multiple_targets()

True

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()